## Char-RNN

In [1]:
import pandas as pd
import numpy as np
import tensorflow.compat.v1 as tf
import keras
from tensorflow.keras import backend
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.models import Sequential
tf.compat.v1.enable_eager_execution()
tf.compat.v1.disable_v2_behavior()
import tensorflow.experimental.numpy as tnp
tf.enable_eager_execution()

Using TensorFlow backend.


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
lyrics = open("Lyrics_Counterparts.txt", encoding="utf-8").read().lower()

In [3]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(lyrics)

In [4]:
tokenizer.texts_to_sequences(["First"])

[[17, 5, 8, 9, 3]]

In [5]:
max_id = len(tokenizer.word_index) #number of distinct characters in text
dataset_size = tokenizer.document_count #total number of characters
print(max_id, " ", dataset_size)

52   71707


In [6]:
[encoded] = np.array(tokenizer.texts_to_sequences([lyrics])) - 1
#encodes the full text so each character is represented by its ID

In [7]:
#splitting dataset for training/test/validation
train_size = dataset_size * 90//100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [8]:
#convert long string of characters into windows for training(batches)
n_steps = 100
window_length = n_steps + 1 #target character = input shifted 1 char ahead
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [9]:
#flatten the dataset for training
dataset = dataset.flat_map(lambda window: window.batch(window_length))
#this gives us a single tensor for each window

In [10]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [11]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [12]:
dataset = dataset.prefetch(1)

In [13]:
model = tf.keras.models.Sequential([
    tf.keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                    dropout=0.2, recurrent_dropout=0.2),
    tf.keras.layers.GRU(128, return_sequences=True, 
                     dropout=0.2, recurrent_dropout=0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(max_id,
                                                   activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=30)

Epoch 1/30
2014/2014 [==============================] - 795s 390ms/step - loss: 2.2605
Epoch 2/30
2014/2014 [==============================] - 688s 341ms/step - loss: 1.5547
Epoch 3/30
2014/2014 [==============================] - 770s 382ms/step - loss: 1.4414
Epoch 4/30
2014/2014 [==============================] - 753s 373ms/step - loss: 1.3881
Epoch 5/30
2014/2014 [==============================] - 675s 334ms/step - loss: 1.3518
Epoch 6/30
2014/2014 [==============================] - 668s 331ms/step - loss: 1.3267
Epoch 7/30
2014/2014 [==============================] - 667s 331ms/step - loss: 1.3086
Epoch 8/30
2014/2014 [==============================] - 663s 329ms/step - loss: 1.2927
Epoch 9/30
2014/2014 [==============================] - 662s 328ms/step - loss: 1.2795
Epoch 10/30
2014/2014 [==============================] - 684s 339ms/step - loss: 1.2684
Epoch 11/30
2014/2014 [==============================] - 714s 353ms/step - loss: 1.2605
Epoch 12/30
2014/2014 [==================

In [32]:
model.save('char_rnn_lyrics')

In [33]:
tf.executing_eagerly()

True

In [34]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [35]:
X_new = preprocess(["How are yo"])
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 1st sentence, last char

InvalidArgumentError:  Specified a list with shape [32,52] from a tensor with shape [1,52]
	 [[node sequential_1/gru_2/TensorArrayUnstack/TensorListFromTensor (defined at <ipython-input-35-f85cbe487a4c>:2) ]] [Op:__inference_predict_function_96915]

Function call stack:
predict_function


In [36]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new,steps=1)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba)/ temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]
#this function picks the next character randomly, with a probability equal to the estimated probability,using tf.random.categorical
#categorical() samples random class indices, given the class probs (logits)
#temperature is a variables that controls the diversity of the characters generated
# a near-zero temperature will output characters with high probabilities
# a high temperature will give all characters equal probability

In [37]:
#this function repeatedly calls the above function to generate text
def complete_text(text, n_chars=100, temperature=1):
    for _ in range(n_chars):
        text += next_char (text, temperature)
    return text

In [38]:
print(complete_text("a", temperature=1))

ValueError: in user code:

    C:\Users\mitch\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py:1478 predict_function  *
        return step_function(self, iterator)
    C:\Users\mitch\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\mitch\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\mitch\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\mitch\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\m

In [21]:
print(complete_text("b", temperature=1))

blindy strength around and sink my way
i watched the come to find the mind
our past payment will bein


In [22]:
print(complete_text("f", temperature=1))

fent portunity
i want to do i will be remembered
we will present, the heart but i know i’m struggling


## Stateful RNN

In [39]:
n_steps = 100
window_length = n_steps + 1 # target = input shifted 1 character ahead
dataset = dataset.repeat().window(window_length, shift=n_steps, drop_remainder=True)

In [40]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.repeat().batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [41]:
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.repeat().map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [42]:
model = tf.keras.models.Sequential([
    tf.keras.layers.GRU(128, return_sequences=True, stateful=True,
                    dropout=0.2, recurrent_dropout=0.2, batch_input_shape=[batch_size, None, max_id]),
    tf.keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(max_id,
                                                   activation="softmax"))
])

In [43]:
class ResetStatesCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [44]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
steps_per_epoch = train_size // batch_size // n_steps

In [45]:
history = model.fit(dataset, epochs=50, callbacks=[ResetStatesCallback()], steps_per_epoch=steps_per_epoch)

Epoch 1/50
20/20 [==============================] - 9s 308ms/step - loss: 3.7607
Epoch 2/50
20/20 [==============================] - 6s 306ms/step - loss: 3.0480
Epoch 3/50
20/20 [==============================] - 6s 291ms/step - loss: 2.9705
Epoch 4/50
20/20 [==============================] - 6s 287ms/step - loss: 2.9200
Epoch 5/50
20/20 [==============================] - 6s 278ms/step - loss: 2.8213
Epoch 6/50
20/20 [==============================] - 6s 299ms/step - loss: 2.6938
Epoch 7/50
20/20 [==============================] - 6s 321ms/step - loss: 2.5960
Epoch 8/50
20/20 [==============================] - 6s 320ms/step - loss: 2.5322
Epoch 9/50
20/20 [==============================] - 6s 318ms/step - loss: 2.4778
Epoch 10/50
20/20 [==============================] - 6s 317ms/step - loss: 2.4455
Epoch 11/50
20/20 [==============================] - 6s 312ms/step - loss: 2.3999
Epoch 12/50
20/20 [==============================] - 6s 315ms/step - loss: 2.3751
Epoch 13/50
20/20 [======

In [46]:
model.save('stateful_rnn_lyrics')

In [48]:
tf.executing_eagerly()

True

In [49]:
#stateless copy to allow us to use different batch sizes
stateless_model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

AttributeError: module 'tensorflow' has no attribute 'get_default_graph'

In [ ]:
stateless_model.build(tf.TensorShape([None, None, max_id]))

In [ ]:
stateless_model.set_weights(model.get_weights())
model = stateless_model

In [ ]:
print(complete_text("t"))